# Exercise of the 8th of october 2020

following "student statistics.csv" and "1-Reminders of Probability Theory" on extradoc

## Re-arrange data

![Ex1](1.PNG)

In [15]:
import pandas as pd
import numpy as np

In [28]:
print("We decided : Glasses:True when Y, Origin:True when Europe, Gender:True when Male, Location:True when France")
df = pd.DataFrame(
    [
        [False, False, True, True],
        [True, False, False, True],
        [True, False, False, True],
        [True, False, True, False],
        [False, True, True, True],
        [True, True, True, True],
        [False, False, True, True],
        [True, False, True, False],
        [True, False, False, False],
        [False, False, False, False],
        [True, False, True, False],
        [False, True, True, True],
        [True, True, False, True],
        [True, False, True, True],
        [False, True, True, True],
    ],
    columns=["Glasses", "Origin", "Gender", "Location"]
)
df.head()

We decided : Glasses:True when Y, Origin:True when Europe, Gender:True when Male, Location:True when France


,Glasses,Origin,Gender,Location
0,False,False,True,True
1,True,False,False,True
2,True,False,False,True
3,True,False,True,False
4,False,True,True,True


## Compute probabilities

![Ex1](2.PNG)

### P(Glasses, Origin, Gender, Location)

In [70]:
def createDistributionDF(df):
    # create array
    probabilities = []
    for i in range(2**df.shape[1]):
        temp = []
        for j in range(df.shape[1]+1):
            temp.append(0)
        probabilities.append(temp)
    # create columns
    for i in range(df.shape[1]):
        variable = True
        counter = 0
        for row in range(len(probabilities)):
            probabilities[row][i] = variable
            counter+=1
            if counter == 2**(df.shape[1]-(i+1)):
                counter = 0
                variable = not variable
    # fill probabilities
    for i in range(len(probabilities)):
        for row in range(df.shape[0]):
            if df.iloc[row, :].tolist() == probabilities[i][:-1]:
                probabilities[i][-1] += 1
        probabilities[i][-1] /= df.shape[0]
    return pd.DataFrame(probabilities, columns=df.columns.tolist()+["prob"])

In [71]:
P_gl_o_ge_l = createDistributionDF(df)
P_gl_o_ge_l

,Glasses,Origin,Gender,Location,prob
0,True,True,True,True,0.066667
1,True,True,True,False,0.000000
2,True,True,False,True,0.066667
3,True,True,False,False,0.000000
4,True,False,True,True,0.066667
5,True,False,True,False,0.200000
6,True,False,False,True,0.133333
7,True,False,False,False,0.066667
8,False,True,True,True,0.200000
9,False,True,True,False,0.000000


### P(Glasses, Origin, Gender)

#### By Marginalization

We get rid of the unused variable and sum on the other on :
Example :

P(Gl,O, Ge) = P(Gl, O, Ge, Lo) + P(Gl, O, Ge, not(Lo))

In [88]:
def marginalization(df, columns):
    # create array
    probabilities = []
    for i in range(2**len(columns)):
        temp = []
        for j in range(len(columns)+1):
            temp.append(0)
        probabilities.append(temp)
    # create columns
    for i in range(len(columns)):
        variable = True
        counter = 0
        for row in range(len(probabilities)):
            probabilities[row][i] = variable
            counter+=1
            if counter == 2**(len(columns)-(i+1)):
                counter = 0
                variable = not variable
    # fill probabilities
    for i in range(len(probabilities)):
        for row in range(df.shape[0]):
            if df.loc[row, columns].tolist() == probabilities[i][:-1]:
                probabilities[i][-1] += df.iloc[row, -1]
    return pd.DataFrame(probabilities, columns=columns+["prob"])

In [89]:
P_gl_o_ge = marginalization(P_gl_o_ge_l, ["Glasses", "Origin", "Gender"])
P_gl_o_ge

,Glasses,Origin,Gender,prob
0,True,True,True,0.066667
1,True,True,False,0.066667
2,True,False,True,0.266667
3,True,False,False,0.200000
4,False,True,True,0.200000
5,False,True,False,0.000000
6,False,False,True,0.133333
7,False,False,False,0.066667


#### By using previous method

In [90]:
P_gl_o_ge = createDistributionDF(df.iloc[:,:3])
P_gl_o_ge

,Glasses,Origin,Gender,prob
0,True,True,True,0.066667
1,True,True,False,0.066667
2,True,False,True,0.266667
3,True,False,False,0.200000
4,False,True,True,0.200000
5,False,True,False,0.000000
6,False,False,True,0.133333
7,False,False,False,0.066667


### P(Location | Glasses, Origin, Gender)

In order to know how many distribution we will get, we need to look at the given number of variables (here 3 : Glasses, Origin, Gender) and put it to the power of the combination possible (here 2<sup>3</sup>)

In [130]:
def createDistributionsDF_given(df_s, variable, given):
    if len(df_s) == 2:
        P_1 = df_s[0]
        P_2 = df_s[1]
    else:
        # get distribution
        P_1 = createDistributionDF(df.loc[:,given+variable])
        P_2 = createDistributionDF(df.loc[:,given])
    probabilities = []
    # create new array
    for i in range(2):
        temp = []
        for j in range(2**len(given)):
            temp.append(0)
        probabilities.append(temp)
    columns = ["" for _ in range(2**len(given))]
    # create column indices
    for i in range(len(given)):
        variable = True
        counter = 0
        for j in range(len(probabilities[0])):
            columns[j] += str(variable)[0]
            counter+=1
            if counter == 2**(len(given)-(i+1)):
                counter = 0
                variable = not variable
    # fill distributions
    for i in range(len(probabilities[0])):
        if P_2.iloc[i,-1] == 0.:
            probabilities[0][i] = "NaN"
            probabilities[1][i] = "NaN"
        else:
            probabilities[0][i] = P_1.iloc[i*2,-1] / P_2.iloc[i,-1]
            probabilities[1][i] = P_1.iloc[(i*2+1),-1] / P_2.iloc[i,-1]
        
    
    return pd.DataFrame(probabilities, columns=columns, index=["True", "False"])

In [131]:
P_l_given_gl_o_ge = createDistributionsDF_given(df, ["Location"], ["Glasses", "Origin", "Gender"])
P_l_given_gl_o_ge

,TTT,TTF,TFT,TFF,FTT,FTF,FFT,FFF
True,1.0,1.0,0.25,0.666667,1.0,NaN,1.0,0.0
False,0.0,0.0,0.75,0.333333,0.0,NaN,0.0,1.0


In [132]:
P_l_given_gl_o_ge = createDistributionsDF_given([P_gl_o_ge_l, P_gl_o_ge], ["Location"], ["Glasses", "Origin", "Gender"])
P_l_given_gl_o_ge

,TTT,TTF,TFT,TFF,FTT,FTF,FFT,FFF
True,1.0,1.0,0.25,0.666667,1.0,NaN,1.0,0.0
False,0.0,0.0,0.75,0.333333,0.0,NaN,0.0,1.0


### P(Glasses, Origin)

In [128]:
P_gl_o = marginalization(P_gl_o_ge_l, ["Glasses", "Origin"])
P_gl_o

,Glasses,Origin,prob
0,True,True,0.133333
1,True,False,0.466667
2,False,True,0.200000
3,False,False,0.200000


In [134]:
P_gl_o = createDistributionDF(df.loc[:,["Glasses", "Origin"]])
P_gl_o

,Glasses,Origin,prob
0,True,True,0.133333
1,True,False,0.466667
2,False,True,0.200000
3,False,False,0.200000


### P(Gender | Glasses, Origin)

In [133]:
P_ge_given_gl_o = createDistributionsDF_given([P_gl_o_ge, P_gl_o], ["Gender"], ["Glasses", "Origin"])
P_ge_given_gl_o

,TT,TF,FT,FF
True,0.5,0.571429,1.0,0.666667
False,0.5,0.428571,0.0,0.333333


In [135]:
P_ge_given_gl_o = createDistributionsDF_given(df, ["Gender"], ["Glasses", "Origin"])
P_ge_given_gl_o

,TT,TF,FT,FF
True,0.5,0.571429,1.0,0.666667
False,0.5,0.428571,0.0,0.333333


### P(Glasses)

In [136]:
P_gl = marginalization(P_gl_o_ge_l, ["Glasses"])
P_gl

,Glasses,prob
0,True,0.6
1,False,0.4


In [137]:
P_gl = createDistributionDF(df.loc[:,["Glasses"]])
P_gl

,Glasses,prob
0,True,0.6
1,False,0.4


### P(Origin | Glasses)

In [139]:
P_o_given_gl = createDistributionsDF_given([P_gl_o, P_gl], ["Origin"], ["Glasses"])
P_o_given_gl

,T,F
True,0.222222,0.5
False,0.777778,0.5


In [140]:
P_o_given_gl = createDistributionsDF_given(df, ["Origin"], ["Glasses"])
P_o_given_gl

,T,F
True,0.222222,0.5
False,0.777778,0.5


## Chain Rule

![Ex1](3.PNG)

P(Gl, O, Ge, L) = P(Gl) * P(O | Gl) * P(Ge | Gl, O) * P(L | Gl, O, Ge)

* ⇔ P(Gl, O, Ge, L) = P(Gl) * (P(Gl, O) / P(Gl)) * (P(Gl, O, Ge) / P(Gl, O)) * (P(Gl, O, Ge, L) / P(Gl, O, Ge))
* ⇔ P(Gl, O, Ge, L) = P(Gl, O) * (P(Gl, O, Ge) / P(Gl, O)) * (P(Gl, O, Ge, L) / P(Gl, O, Ge))
* ⇔ P(Gl, O, Ge, L) = P(Gl, O, Ge) * (P(Gl, O, Ge, L) / P(Gl, O, Ge))
* ⇔ P(Gl, O, Ge, L) = P(Gl, O, Ge, L)

True

## Chain Rule Changes

![Ex1](4.PNG)

If O ⊥ Gl Then it means that P(O|Gl) = P(O)

(in our example there were not since 7/9 != 10/15)

Thus, it will change the following formula :

* P(Gl, O, Ge, L) = P(Gl) * P(O | Gl) * P(Ge | Gl, O) * P(L | Gl, O, Ge)
* ⇔ P(Gl, O, Ge, L) = P(Gl) * P(O) * P(Ge | Gl, O) * P(L | Gl, O, Ge)

If (L ⊥ O,Ge) | Gl Then it means that P(L | Gl, O, Ge) = P(L | Gl)

Thus, it will change the following formula :

* P(Gl, O, Ge, L) = P(Gl) * P(O) * P(Ge | Gl, O) * P(L | Gl, O, Ge)
* ⇔ P(Gl, O, Ge, L) = P(Gl) * P(O) * P(Ge | Gl, O) * P(L | Gl)

Hence, since we need to store : P(Gl, O, Ge, L) = P(Gl) * P(O | Gl) * P(Ge | Gl, O) * P(L | Gl, O, Ge)
* ((2<sup>1</sup>-1) + (2<sup>1</sup>-1)×2<sup>1</sup> + (2<sup>1</sup>-1)×2<sup>2</sup> + (2<sup>1</sup>-1)×2<sup>4</sup>)
* ⇔ (1 + 2 + 4 + 8)
* ⇔ 15 (complexity storage values)

But with the new formula : P(Gl, O, Ge, L) = P(Gl) * P(O) * P(Ge | Gl, O) * P(L | Gl)
* ((2<sup>1</sup>-1) + (2<sup>1</sup>-1) + (2<sup>1</sup>-1)×2<sup>2</sup>  + (2<sup>1</sup>-1)×2<sup>1</sup>)
* ⇔ (1 + 1 + 4 + 2)
* ⇔ 8 (complexity storage values)